In [1]:
!pip install pyswarms

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 3.0 MB/s eta 0:00:00


In [2]:


import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import pyswarms as ps
from pyswarms.utils.functions.single_obj import sphere



In [3]:
df = pd.read_csv("/kaggle/input/cloud-computing-performance-metrics/vmCloud_data.csv")

df.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,vm_id,timestamp,cpu_usage,memory_usage,network_traffic,power_consumption,num_executed_instructions,execution_time,energy_efficiency,task_type,task_priority,task_status
0,c5215826-6237-4a33-9312-72c1df909881,2023-01-25 09:10:54,54.881350,78.950861,164.775973,287.808986,7527.0,69.345575,0.553589,network,medium,waiting
1,29690bc6-1f34-403b-b509-a1ecb1834fb8,2023-01-26 04:46:34,71.518937,29.901883,NaN,362.273569,5348.0,41.396040,0.349856,io,high,completed
2,2e55abc3-5bad-46cb-b445-a577f5e9bf2a,2023-01-13 23:39:47,NaN,92.709195,203.674847,231.467903,5483.0,24.602549,0.796277,io,medium,completed
3,e672e32f-c134-4fbc-992b-34eb63bef6bf,2023-02-09 11:45:49,54.488318,88.100960,NaN,195.639954,5876.0,16.456670,0.529511,compute,high,completed
4,f38b8b50-6926-4533-be4f-89ad11624071,2023-06-14 08:27:26,42.365480,NaN,NaN,359.451537,3361.0,55.307992,0.351907,NaN,medium,waiting


In [4]:
df.columns

Index(['vm_id', 'timestamp', 'cpu_usage', 'memory_usage', 'network_traffic',
       'power_consumption', 'num_executed_instructions', 'execution_time',
       'energy_efficiency', 'task_type', 'task_priority', 'task_status'],
      dtype='object')

In [5]:
# Data Cleaning
def clean_data(df):
    # Drop irrelevant columns
    df = df.drop(columns=['vm_id','timestamp'])

    
    # Handle missing values
    # Numerical columns: impute with median
    num_cols = df.select_dtypes(include=np.number).columns
    num_imputer = SimpleImputer(strategy='median')
    df[num_cols] = num_imputer.fit_transform(df[num_cols])
    
    # Categorical columns: impute with mode
    cat_cols = df.select_dtypes(include='object').columns.drop('task_status')
    cat_imputer = SimpleImputer(strategy='most_frequent')
    df[cat_cols] = cat_imputer.fit_transform(df[cat_cols])
    
    # Remove duplicates
    df = df.drop_duplicates()
    
    return df

cleaned_df = clean_data(df)
cleaned_df.head()


,cpu_usage,memory_usage,network_traffic,power_consumption,num_executed_instructions,execution_time,energy_efficiency,task_type,task_priority,task_status
0,54.881350,78.950861,164.775973,287.808986,7527.0,69.345575,0.553589,network,medium,waiting
1,71.518937,29.901883,500.007595,362.273569,5348.0,41.396040,0.349856,io,high,completed
2,50.054758,92.709195,203.674847,231.467903,5483.0,24.602549,0.796277,io,medium,completed
3,54.488318,88.100960,500.007595,195.639954,5876.0,16.456670,0.529511,compute,high,completed
4,42.365480,49.976089,500.007595,359.451537,3361.0,55.307992,0.351907,io,medium,waiting


In [6]:
cleaned_df['energy_efficiency'] = cleaned_df['energy_efficiency'] * 100

cleaned_df.head()


,cpu_usage,memory_usage,network_traffic,power_consumption,num_executed_instructions,execution_time,energy_efficiency,task_type,task_priority,task_status
0,54.881350,78.950861,164.775973,287.808986,7527.0,69.345575,55.358901,network,medium,waiting
1,71.518937,29.901883,500.007595,362.273569,5348.0,41.396040,34.985621,io,high,completed
2,50.054758,92.709195,203.674847,231.467903,5483.0,24.602549,79.627697,io,medium,completed
3,54.488318,88.100960,500.007595,195.639954,5876.0,16.456670,52.951078,compute,high,completed
4,42.365480,49.976089,500.007595,359.451537,3361.0,55.307992,35.190726,io,medium,waiting


In [7]:
cleaned_df = cleaned_df.drop(columns=['num_executed_instructions'])
cleaned_df = cleaned_df.drop(columns=['task_type','task_priority'])

cleaned_df.head()


,cpu_usage,memory_usage,network_traffic,power_consumption,execution_time,energy_efficiency,task_status
0,54.881350,78.950861,164.775973,287.808986,69.345575,55.358901,waiting
1,71.518937,29.901883,500.007595,362.273569,41.396040,34.985621,completed
2,50.054758,92.709195,203.674847,231.467903,24.602549,79.627697,completed
3,54.488318,88.100960,500.007595,195.639954,16.456670,52.951078,completed
4,42.365480,49.976089,500.007595,359.451537,55.307992,35.190726,waiting


In [8]:
# Preprocessing
def preprocess_data(df, target_column='task_status'):
    # Convert categorical columns
    le = LabelEncoder()
    for col in df.select_dtypes(include='object').columns:
        if col != target_column:
            df[col] = le.fit_transform(df[col])
    
    # Encode target variable
    y = le.fit_transform(df[target_column])
    X = df.drop(columns=[target_column])
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Scale features
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, X_test, y_train, y_test, X.shape[1]

X_train, X_test, y_train, y_test, n_features = preprocess_data(cleaned_df)



In [9]:
# LSTM Model Preparation
def reshape_for_lstm(X_train, X_test):
    X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
    return X_train, X_test

X_train_lstm, X_test_lstm = reshape_for_lstm(X_train, X_test)



In [10]:
# Modified LSTM Model
def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(128, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(64))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

lstm_model = create_lstm_model((X_train_lstm.shape[1], X_train_lstm.shape[2]))
history = lstm_model.fit(X_train_lstm, y_train, 
                        epochs=100, 
                        batch_size=64,
                        validation_split=0.2,
                        verbose=1)



/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 98s 5ms/step - accuracy: 0.3005 - loss: -45.4481 - val_accuracy: 0.3003 - val_loss: -169.3988
Epoch 2/100
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 91s 5ms/step - accuracy: 0.3000 - loss: -212.3144 - val_accuracy: 0.3003 - val_loss: -334.4058
Epoch 3/100
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 92s 5ms/step - accuracy: 0.2991 - loss: -376.1619 - val_accuracy: 0.3003 - val_loss: -499.4401
Epoch 4/100
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 93s 5ms/step - accuracy: 0.3004 - loss: -537.1557 - val_accuracy: 0.3003 - val_loss: -664.0661
Epoch 5/100
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 93s 5ms/step - accuracy: 0.3009 - loss: -703.2362 - val_accuracy: 0.3003 - val_loss: -829.0229
Epoch 6/100
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 93s 5ms/step - accuracy: 0.3001 - loss: -877.1121 - val_accuracy: 0.3003 - val_loss: -994.2122
Epoch 7/100
20000/20000 ━━━━━━━━━━━━━━━━━━━━ 93s 5ms/step - accuracy: 0.3003 - loss: -1039.0278 - val_accuracy: 0.3003 - val_loss: -1159.2346
Epoch 8/100
20000/2

In [11]:
# LSTM Evaluation
lstm_pred = (lstm_model.predict(X_test_lstm) > 0.5).astype(int)
print("LSTM Classification Report:")
print(classification_report(y_test, lstm_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, lstm_pred))



12500/12500 ━━━━━━━━━━━━━━━━━━━━ 19s 1ms/step
LSTM Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00    119845
           1       0.30      1.00      0.46    119917
           2       0.00      0.00      0.00    120400
           3       0.00      0.00      0.00     39838

    accuracy                           0.30    400000
   macro avg       0.07      0.25      0.12    400000
weighted avg       0.09      0.30      0.14    400000

Confusion Matrix:
[[     0 119845      0      0]
 [     0 119917      0      0]
 [     0 120400      0      0]
 [     0  39838      0      0]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
